<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/evaluation/answer_and_context_relevancy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Answer Relevancy and Context Relevancy Evaluations

In this notebook, we demonstrate how to utilize the `AnswerRelevancyEvaluator` and `ContextRelevancyEvaluator` classes to get a measure on the relevancy of a generated answer and retrieved contexts, respectively, to a given user query. Both of these evaluators return a `score` that is between 0 and 1 as well as a generated `feedback` explaining the score. Note that, higher score means higher relevancy. In particular, we prompt the judge LLM to take a step-by-step approach in providing a relevancy score, asking it to answer the following two questions of a generated answer to a query for answer relevancy (for context relevancy these are slightly adjusted):

1. Does the provided response match the subject matter of the user's query?
2. Does the provided response attempt to address the focus or perspective on the subject matter taken on by the user's query?

Each question is worth 1 point and so a perfect evaluation would yield a score of 2/2.

In [ ]:
%pip install llama-index-llms-openai

In [ ]:
import nest_asyncio
from tqdm.asyncio import tqdm_asyncio

nest_asyncio.apply()

In [ ]:
def displayify_df(df):
    """For pretty displaying DataFrame in a notebook."""
    display_df = df.style.set_properties(
        **{
            "inline-size": "300px",
            "overflow-wrap": "break-word",
        }
    )
    display(display_df)

### Download the dataset (`LabelledRagDataset`)

For this demonstration, we will use a llama-dataset provided through our [llama-hub](https://llamahub.ai).

In [ ]:
from llama_index.core.llama_dataset import download_llama_dataset
from llama_index.core.llama_pack import download_llama_pack
from llama_index.core import VectorStoreIndex

# download and install dependencies for benchmark dataset
rag_dataset, documents = download_llama_dataset(
    "EvaluatingLlmSurveyPaperDataset", "./data"
)

In [ ]:
rag_dataset.to_pandas()[:5]

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,What are the potential risks associated with l...,[Evaluating Large Language Models: A\nComprehe...,"According to the context information, the pote...",ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
1,How does the survey categorize the evaluation ...,[Evaluating Large Language Models: A\nComprehe...,The survey categorizes the evaluation of LLMs ...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
2,What are the different types of reasoning disc...,[Contents\n1 Introduction 4\n2 Taxonomy and Ro...,The different types of reasoning discussed in ...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
3,How is toxicity evaluated in language models a...,[Contents\n1 Introduction 4\n2 Taxonomy and Ro...,Toxicity is evaluated in language models accor...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
4,"In the context of specialized LLMs evaluation,...",[5.1.3 Alignment Robustness . . . . . . . . . ...,"In the context of specialized LLMs evaluation,...",ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)


Next, we build a RAG over the same source documents used to created the `rag_dataset`.

In [ ]:
index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()

With our RAG (i.e `query_engine`) defined, we can make predictions (i.e., generate responses to the query) with it over the `rag_dataset`.

In [ ]:
prediction_dataset = await rag_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=100, show_progress=True
)

Batch processing of predictions: 100%|██████████████████████| 76/76 [00:06<00:00, 10.93it/s]


### Evaluating Answer and Context Relevancy Separately

We first need to define our evaluators (i.e. `AnswerRelevancyEvaluator` & `ContextRelevancyEvaluator`):

In [ ]:
# instantiate the gpt-4 judges
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import (
    AnswerRelevancyEvaluator,
    ContextRelevancyEvaluator,
)

judges = {}

judges["answer_relevancy"] = AnswerRelevancyEvaluator(
    llm=OpenAI(temperature=0, model="gpt-3.5-turbo"),
)

judges["context_relevancy"] = ContextRelevancyEvaluator(
    llm=OpenAI(temperature=0, model="gpt-4"),
)

Now, we can use our evaluator to make evaluations by looping through all of the <example, prediction> pairs.

In [ ]:
eval_tasks = []
for example, prediction in zip(
    rag_dataset.examples, prediction_dataset.predictions
):
    eval_tasks.append(
        judges["answer_relevancy"].aevaluate(
            query=example.query,
            response=prediction.response,
            sleep_time_in_seconds=1.0,
        )
    )
    eval_tasks.append(
        judges["context_relevancy"].aevaluate(
            query=example.query,
            contexts=prediction.contexts,
            sleep_time_in_seconds=1.0,
        )
    )

In [ ]:
eval_results1 = await tqdm_asyncio.gather(*eval_tasks[:250])

100%|█████████████████████████████████████████████████████| 250/250 [00:28<00:00,  8.85it/s]


In [ ]:
eval_results2 = await tqdm_asyncio.gather(*eval_tasks[250:])

100%|█████████████████████████████████████████████████████| 302/302 [00:31<00:00,  9.62it/s]


In [ ]:
eval_results = eval_results1 + eval_results2

In [ ]:
evals = {
    "answer_relevancy": eval_results[::2],
    "context_relevancy": eval_results[1::2],
}

### Taking a look at the evaluation results

Here we use a utility function to convert the list of `EvaluationResult` objects into something more notebook friendly. This utility will provide two DataFrames, one deep one containing all of the evaluation results, and another one which aggregates via taking the mean of all the scores, per evaluation method.

In [ ]:
from llama_index.core.evaluation.notebook_utils import get_eval_results_df
import pandas as pd

deep_dfs = {}
mean_dfs = {}
for metric in evals.keys():
    deep_df, mean_df = get_eval_results_df(
        names=["baseline"] * len(evals[metric]),
        results_arr=evals[metric],
        metric=metric,
    )
    deep_dfs[metric] = deep_df
    mean_dfs[metric] = mean_df

In [ ]:
mean_scores_df = pd.concat(
    [mdf.reset_index() for _, mdf in mean_dfs.items()],
    axis=0,
    ignore_index=True,
)
mean_scores_df = mean_scores_df.set_index("index")
mean_scores_df.index = mean_scores_df.index.set_names(["metrics"])
mean_scores_df

rag,baseline
metrics,
mean_answer_relevancy_score,0.914855
mean_context_relevancy_score,0.572273


The above utility also provides the mean score across all of the evaluations in `mean_df`.

We can get a look at the raw distribution of the scores by invoking `value_counts()` on the `deep_df`.

In [ ]:
deep_dfs["answer_relevancy"]["scores"].value_counts()

scores
1.0    250
0.0     21
0.5      5
Name: count, dtype: int64

In [ ]:
deep_dfs["context_relevancy"]["scores"].value_counts()

scores
1.000    89
0.000    70
0.750    49
0.250    23
0.625    14
0.500    11
0.375    10
0.875     9
Name: count, dtype: int64

It looks like for the most part, the default RAG does fairly well in terms of generating answers that are relevant to the query. Getting a closer look is made possible by viewing the records of any of the `deep_df`'s.

In [ ]:
displayify_df(deep_dfs["context_relevancy"].head(2))

And, of course you can apply any filters as you like. For example, if you want to look at the examples that yielded less than perfect results.

In [ ]:
cond = deep_dfs["context_relevancy"]["scores"] < 1
displayify_df(deep_dfs["context_relevancy"][cond].head(5))